In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

In [3]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(0.13,0.30)])
##下载训练集和数据集
train_data=datasets.MNIST(root='E:\pycharm\pytorch-practice.\data', train=True, transform=transform, download=True)
test_data=datasets.MNIST(root='E:\pycharm\pytorch-practice.\data', train=False, transform=transform, download=True)
train_loader=DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader=DataLoader(dataset=test_data, batch_size=64, shuffle=False)

In [4]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.net=nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.Dropout1d(0.5),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,10),
        )
    def forward(self,x):
        y=self.net(x)
        return y
        

In [5]:
loss_fn=nn.CrossEntropyLoss()
model=DNN()
model=model.to('cuda:0')
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=0.5)



In [24]:
##训练网络
epochs=5
losses=[]
for epoch in range(epochs):
    for (x,y) in train_loader:
        x,y=x.to('cuda:0'),y.to('cuda:0')
        pred=model(x)
        loss=loss_fn(pred,y)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
fig=plt.figure(figsize=(10,6))
plt.plot(range(len(losses)),losses)
plt.show()

D:\anaconda\envs\gpupytorch\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (10) must match the size of tensor b (64) at non-singleton dimension 1

In [22]:
##测试网络
correct=0
total=0

with torch.no_grad():
    for (x,y) in test_loader:
        x,y=x.to('cuda:0'),y.to('cuda:0')
        pred=model(x)
        _,predicted=torch.max(pred,dim=1)
        correct+=torch.sum((predicted==y))
        total+=y.size(0)
print(f"测试集精度:{100*correct/total:.2f}%")

测试集精度:97.94%


In [18]:
pred=torch.tensor([[0.1,0.3,0.6],
                   [0.2,0.4,0.7],
                   [0.1,0.5,0.9]])

max,predicted=torch.max(pred,dim=1)
print(max,predicted)

tensor([0.6000, 0.7000, 0.9000]) tensor([2, 2, 2])
